<a href="https://colab.research.google.com/github/rabeyamily/projects_hw01/blob/main/mily_projects_homework01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import userdata
import requests
import datetime

def get_weather(city):

    #Fetch weather data for a given city using the WeatherStack API.

    api_key = userdata.get("weatherstack_API")
    url = f"http://api.weatherstack.com/current?access_key={api_key}&query={city}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        if 'current' in data:
            return data
        else:
            print(f"Error: 'current' data not found in response for {city}.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

def get_events(city, start_date, end_date):

    #Fetch event data for a given city and date range using the Ticketmaster API.

    api_key = userdata.get("ticketmaster_API")
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    params = {
        "apikey": api_key,
        "city": city,
        "startDateTime": start_date,
        "endDateTime": end_date,
        "size": 5  # Limit to 5 events
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching events: {e}")
        return None

def weather_based_event_recommender(city):

    #Recommend events based on current weather conditions for a given city.

    # Fetch weather data
    weather_data = get_weather(city)
    if not weather_data or 'current' not in weather_data:
        print(f"Unable to fetch weather data for {city}. Please check the city name and try again.")
        return False

    # Extract location information
    location = weather_data.get('location', {})
    city_name = location.get('name', city)
    country_name = location.get('country', 'Unknown')

    # Extract weather information
    temperature = weather_data['current'].get('temperature', 'N/A')
    weather_description = weather_data['current'].get('weather_descriptions', ['N/A'])[0]

    # Display location and weather information
    print("\n" + "="*50)
    print(f"Weather and Events for {city_name}, {country_name}".center(50))
    print("="*50)
    print(f"\nCurrent weather:")
    print(f"Temperature: {temperature}°C")
    print(f"Conditions: {weather_description}")

    # Set date range for event search (next 7 days)
    start_date = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
    end_date = (datetime.datetime.now() + datetime.timedelta(days=7)).strftime("%Y-%m-%dT%H:%M:%SZ")

    # Fetch event data
    events = get_events(city, start_date, end_date)
    if events and '_embedded' in events and 'events' in events['_embedded']:
        print("\nRecommended events:")
        print("-"*50)
        for i, event in enumerate(events['_embedded']['events'], 1):
            name = event['name']
            date = event['dates']['start']['localDate']
            time = event['dates']['start'].get('localTime', 'Time not specified')
            venue = event['_embedded']['venues'][0]['name']
            print(f"Event {i}:")
            print(f"  Name: {name}")
            print(f"  Date: {date}")
            print(f"  Time: {time}")
            print(f"  Venue: {venue}")
            print("-"*50)
    else:
        print("\nNo events found for the next 7 days.")

    return True

#main execution
if __name__ == "__main__":
    while True:
        city = input("Enter a city name: ").strip()
        if weather_based_event_recommender(city):
            break

    print("\nThank you for using the weather-based event recommender.")



Enter a city name: mumbai

       Weather and Events for Mumbai, India       

Current weather:
Temperature: 27°C
Conditions: Mist

No events found for the next 7 days.

Thank you for using the weather-based event recommender.
